In [1]:
import torch
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
class BasicBlock(torch.nn.Module):
    """残差块"""

    def __init__(self, inplanes, planes, stride=1):
        """初始化"""

        super(BasicBlock, self).__init__()

        self.conv1 = torch.nn.Conv2d(in_channels=inplanes, out_channels=planes, kernel_size=(3, 3),
                                     stride=(stride, stride), padding=1)  # 卷积层1
        self.bn1 = torch.nn.BatchNorm2d(planes)  # 标准化层1


        self.conv2 = torch.nn.Conv2d(in_channels=planes, out_channels=planes, kernel_size=(3, 3), padding=1)  # 卷积层2
        self.bn2 = torch.nn.BatchNorm2d(planes)  # 标准化层2

        # 如果步长不为1, 用1*1的卷积实现下采样
        if stride != 1:
            self.downsample = torch.nn.Sequential(
                # 下采样
                torch.nn.Conv2d(in_channels=inplanes, out_channels=planes, kernel_size=(1, 1), stride=(stride, stride)))
        else:
            self.downsample = lambda x: x  # 返回x

    def forward(self, input):
        """前向传播"""
        out = self.conv1(input)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        identity = self.downsample(input)
        output = torch.add(out, identity)
        output = F.relu(output)

        return output


ResNet_18 = torch.nn.Sequential(
    # 初始层
    torch.nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1)),  # 卷积
    torch.nn.BatchNorm2d(64),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((2, 2)),  # 池化

    # 8个block(每个为两层)
    BasicBlock(64, 64, stride=1),
    BasicBlock(64, 64, stride=1),
    BasicBlock(64, 128, stride=2),
    BasicBlock(128, 128, stride=1),
    BasicBlock(128, 256, stride=2),
    BasicBlock(256, 256, stride=1),
    BasicBlock(256, 512, stride=2),
    BasicBlock(512, 512, stride=1),
    torch.nn.AvgPool2d(2),  # 池化

    torch.nn.Flatten(),  # 平铺层

    # 全连接层
    torch.nn.Linear(512, 100)  # 100类
)

In [3]:
def get_data():
    """获取数据"""

    # 获取测试集
    train = torchvision.datasets.CIFAR100(root="./data", train=True, download=True,
                                       transform=torchvision.transforms.Compose([
                                           torchvision.transforms.ToTensor(),  # 转换成张量
                                           torchvision.transforms.Normalize((0.1307,), (0.3081,))  # 标准化
                                       ]))
    train_loader = DataLoader(train, batch_size=batch_size)  # 分割测试集

    # 获取测试集
    test = torchvision.datasets.CIFAR100(root="./data", train=False, download=True,
                                      transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),  # 转换成张量
                                          torchvision.transforms.Normalize((0.1307,), (0.3081,))  # 标准化
                                      ]))
    test_loader = DataLoader(test, batch_size=batch_size)  # 分割训练

    # 返回分割好的训练集和测试集
    return train_loader, test_loader

In [4]:
def train(model, epoch, train_loader):
    """训练"""
    # 训练模式
    model.train()
    # 迭代
    for step, (x, y) in enumerate(train_loader):
        # 加速
        if use_cuda:
            model = model.cuda()
            x, y = x.cuda(), y.cuda()
        # 梯度清零
        optimizer.zero_grad()
        output = model(x)
        # 计算损失
        loss = F.cross_entropy(output, y)
        # 反向传播
        loss.backward()
        # 更新梯度
        optimizer.step()
        # 打印损失
        if step % 10 == 0:
            print('Epoch: {}, Step {}, Loss: {}'.format(epoch, step, loss))

In [5]:
def test(model, test_loader):
    """测试"""
    # 测试模式
    model.eval()
    # 存放正确个数
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            # 加速
            if use_cuda:
                model = model.cuda()
                x, y = x.cuda(), y.cuda()
            # 获取结果
            output = model(x)
            # 预测结果
            pred = output.argmax(dim=1, keepdim=True)
            # 计算准确个数
            correct += pred.eq(y.view_as(pred)).sum().item()
    # 计算准确率
    accuracy = correct / len(test_loader.dataset) * 100
    # 输出准确
    print("Test Accuracy: {}%".format(accuracy))

In [6]:
# 定义超参数
batch_size = 2048  # 一次训练的样本数目
learning_rate = 0.001  # 学习率
iteration_num = 500  # 迭代次数
network = ResNet_18
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)  # 优化器
# GPU 加速
use_cuda = torch.cuda.is_available()
if use_cuda:
    network.cuda()
print("是否使用 GPU 加速:", use_cuda)

是否使用 GPU 加速: True


In [7]:
def main():
    # 获取数据
    train_loader, test_loader = get_data()

    # 迭代
    for epoch in range(iteration_num):
        print("\n================ epoch: {} ================".format(epoch))
        train(network, epoch, train_loader)
        test(network, test_loader)

if __name__ == "__main__":
    main()

Files already downloaded and verified
Files already downloaded and verified

================ epoch: 0 ================
Epoch: 0, Step 0, Loss: 4.728081226348877
Epoch: 0, Step 10, Loss: 4.1027936935424805
Epoch: 0, Step 20, Loss: 3.6873135566711426
Test Accuracy: 8.39%

================ epoch: 1 ================
Epoch: 1, Step 0, Loss: 3.667262077331543
Epoch: 1, Step 10, Loss: 3.4072015285491943
Epoch: 1, Step 20, Loss: 3.1144373416900635
Test Accuracy: 19.52%

================ epoch: 2 ================
Epoch: 2, Step 0, Loss: 3.114158868789673
Epoch: 2, Step 10, Loss: 2.885533094406128
Epoch: 2, Step 20, Loss: 2.6425836086273193
Test Accuracy: 28.22%

================ epoch: 3 ================
Epoch: 3, Step 0, Loss: 2.630603075027466
Epoch: 3, Step 10, Loss: 2.423150062561035
Epoch: 3, Step 20, Loss: 2.23103666305542
Test Accuracy: 29.830000000000002%

================ epoch: 4 ================
Epoch: 4, Step 0, Loss: 2.268803596496582
Epoch: 4, Step 10, Loss: 2.090759038925171
Epo

Epoch: 39, Step 0, Loss: 0.00024522736202925444
Epoch: 39, Step 10, Loss: 0.0002484918513800949
Epoch: 39, Step 20, Loss: 0.001692371559329331
Test Accuracy: 48.79%

================ epoch: 40 ================
Epoch: 40, Step 0, Loss: 0.00023169787891674787
Epoch: 40, Step 10, Loss: 0.00023768868413753808
Epoch: 40, Step 20, Loss: 0.001987111521884799
Test Accuracy: 48.88%

================ epoch: 41 ================
Epoch: 41, Step 0, Loss: 0.0002259179309476167
Epoch: 41, Step 10, Loss: 0.00022987666307017207
Epoch: 41, Step 20, Loss: 0.0017290922114625573
Test Accuracy: 48.83%

================ epoch: 42 ================
Epoch: 42, Step 0, Loss: 0.00021398928947746754
Epoch: 42, Step 10, Loss: 0.00021955464035272598
Epoch: 42, Step 20, Loss: 0.0020861183293163776
Test Accuracy: 48.9%

================ epoch: 43 ================
Epoch: 43, Step 0, Loss: 0.00020956028311047703
Epoch: 43, Step 10, Loss: 0.0002135164977516979
Epoch: 43, Step 20, Loss: 0.0016350599471479654
Test Accuracy

Test Accuracy: 48.809999999999995%

================ epoch: 78 ================
Epoch: 78, Step 0, Loss: 8.114404772641137e-05
Epoch: 78, Step 10, Loss: 8.29734854050912e-05
Epoch: 78, Step 20, Loss: 0.0008460275712423027
Test Accuracy: 48.949999999999996%

================ epoch: 79 ================
Epoch: 79, Step 0, Loss: 7.997002103365958e-05
Epoch: 79, Step 10, Loss: 8.248416997957975e-05
Epoch: 79, Step 20, Loss: 0.0008671972318552434
Test Accuracy: 48.809999999999995%

================ epoch: 80 ================
Epoch: 80, Step 0, Loss: 7.782728789607063e-05
Epoch: 80, Step 10, Loss: 7.973724132170901e-05
Epoch: 80, Step 20, Loss: 0.0008293965947814286
Test Accuracy: 48.94%

================ epoch: 81 ================
Epoch: 81, Step 0, Loss: 7.684501906624064e-05
Epoch: 81, Step 10, Loss: 7.953368913149461e-05
Epoch: 81, Step 20, Loss: 0.0008386201807297766
Test Accuracy: 48.77%

================ epoch: 82 ================
Epoch: 82, Step 0, Loss: 7.490565621992573e-05
Epoch: 8

Epoch: 116, Step 0, Loss: 4.216696470393799e-05
Epoch: 116, Step 10, Loss: 4.35721012763679e-05
Epoch: 116, Step 20, Loss: 0.000653571099974215
Test Accuracy: 48.64%

================ epoch: 117 ================
Epoch: 117, Step 0, Loss: 4.186672595096752e-05
Epoch: 117, Step 10, Loss: 4.390458343550563e-05
Epoch: 117, Step 20, Loss: 0.0007261040736921132
Test Accuracy: 48.559999999999995%

================ epoch: 118 ================
Epoch: 118, Step 0, Loss: 4.1119445086224005e-05
Epoch: 118, Step 10, Loss: 4.258465924067423e-05
Epoch: 118, Step 20, Loss: 0.0006248251302167773
Test Accuracy: 48.72%

================ epoch: 119 ================
Epoch: 119, Step 0, Loss: 4.077391713508405e-05
Epoch: 119, Step 10, Loss: 4.2920753912767395e-05
Epoch: 119, Step 20, Loss: 0.0007621983531862497
Test Accuracy: 48.43%

================ epoch: 120 ================
Epoch: 120, Step 0, Loss: 4.015081867692061e-05
Epoch: 120, Step 10, Loss: 4.1717761632753536e-05
Epoch: 120, Step 20, Loss: 0.0006

Test Accuracy: 39.73%

================ epoch: 154 ================
Epoch: 154, Step 0, Loss: 1.2614099979400635
Epoch: 154, Step 10, Loss: 1.1126285791397095
Epoch: 154, Step 20, Loss: 0.9946805238723755
Test Accuracy: 35.949999999999996%

================ epoch: 155 ================
Epoch: 155, Step 0, Loss: 0.8327398300170898
Epoch: 155, Step 10, Loss: 0.7346659898757935
Epoch: 155, Step 20, Loss: 0.8148111701011658
Test Accuracy: 35.02%

================ epoch: 156 ================
Epoch: 156, Step 0, Loss: 0.6735419034957886
Epoch: 156, Step 10, Loss: 0.5975443720817566
Epoch: 156, Step 20, Loss: 0.5263404846191406
Test Accuracy: 39.489999999999995%

================ epoch: 157 ================
Epoch: 157, Step 0, Loss: 0.5397576093673706
Epoch: 157, Step 10, Loss: 0.38939106464385986
Epoch: 157, Step 20, Loss: 0.3063109219074249
Test Accuracy: 40.58%

================ epoch: 158 ================
Epoch: 158, Step 0, Loss: 0.28321465849876404
Epoch: 158, Step 10, Loss: 0.2175882607

...

Test Accuracy: 48.75%

================ epoch: 344 ================
Epoch: 344, Step 0, Loss: 0.00011281203478574753
Epoch: 344, Step 10, Loss: 0.00011664463818306103
Epoch: 344, Step 20, Loss: 0.0006384445005096495
Test Accuracy: 48.72%

================ epoch: 345 ================
Epoch: 345, Step 0, Loss: 0.000105181141407229
Epoch: 345, Step 10, Loss: 0.00010988164285663515
Epoch: 345, Step 20, Loss: 0.000491201935801655
Test Accuracy: 48.77%

================ epoch: 346 ================
Epoch: 346, Step 0, Loss: 0.00010036499588750303
Epoch: 346, Step 10, Loss: 0.00010430287511553615
Epoch: 346, Step 20, Loss: 0.000562732107937336
Test Accuracy: 48.75%

================ epoch: 347 ================
Epoch: 347, Step 0, Loss: 9.413026418769732e-05
Epoch: 347, Step 10, Loss: 9.882693848339841e-05
Epoch: 347, Step 20, Loss: 0.00042751626460812986
Test Accuracy: 48.809999999999995%

================ epoch: 348 ================
Epoch: 348, Step 0, Loss: 9.033134847413749e-05
Epoch: 348, 

Epoch: 382, Step 0, Loss: 3.2519237720407546e-05
Epoch: 382, Step 10, Loss: 3.449025098234415e-05
Epoch: 382, Step 20, Loss: 5.311537461238913e-05
Test Accuracy: 48.980000000000004%

================ epoch: 383 ================
Epoch: 383, Step 0, Loss: 3.170577474520542e-05
Epoch: 383, Step 10, Loss: 3.36485609295778e-05
Epoch: 383, Step 20, Loss: 5.303023863234557e-05
Test Accuracy: 49.07%

================ epoch: 384 ================
Epoch: 384, Step 0, Loss: 3.117241431027651e-05
Epoch: 384, Step 10, Loss: 3.3011729101417586e-05
Epoch: 384, Step 20, Loss: 5.004062404623255e-05
Test Accuracy: 49.0%

================ epoch: 385 ================
Epoch: 385, Step 0, Loss: 3.042953903786838e-05
Epoch: 385, Step 10, Loss: 3.2264841138385236e-05
Epoch: 385, Step 20, Loss: 4.9656544433673844e-05
Test Accuracy: 49.09%

================ epoch: 386 ================
Epoch: 386, Step 0, Loss: 2.9811439162585884e-05
Epoch: 386, Step 10, Loss: 3.1592331652063876e-05
Epoch: 386, Step 20, Loss: 4.7

Test Accuracy: 48.91%

================ epoch: 420 ================
Epoch: 420, Step 0, Loss: 1.535689261800144e-05
Epoch: 420, Step 10, Loss: 1.630140104680322e-05
Epoch: 420, Step 20, Loss: 2.0281324395909905e-05
Test Accuracy: 48.88%

================ epoch: 421 ================
Epoch: 421, Step 0, Loss: 1.511918435426196e-05
Epoch: 421, Step 10, Loss: 1.6040003174566664e-05
Epoch: 421, Step 20, Loss: 2.0006280465167947e-05
Test Accuracy: 48.89%

================ epoch: 422 ================
Epoch: 422, Step 0, Loss: 1.4859242583042942e-05
Epoch: 422, Step 10, Loss: 1.5759223970235325e-05
Epoch: 422, Step 20, Loss: 1.9610333765740506e-05
Test Accuracy: 48.89%

================ epoch: 423 ================
Epoch: 423, Step 0, Loss: 1.4604535863327328e-05
Epoch: 423, Step 10, Loss: 1.5500270819757134e-05
Epoch: 423, Step 20, Loss: 1.9137469280394726e-05
Test Accuracy: 48.9%

================ epoch: 424 ================
Epoch: 424, Step 0, Loss: 1.4344770534080453e-05
Epoch: 424, Step 10

Epoch: 457, Step 20, Loss: 9.896704796119593e-06
Test Accuracy: 48.8%

================ epoch: 458 ================
Epoch: 458, Step 0, Loss: 8.090067240118515e-06
Epoch: 458, Step 10, Loss: 8.620215339760762e-06
Epoch: 458, Step 20, Loss: 9.727362339617684e-06
Test Accuracy: 48.79%

================ epoch: 459 ================
Epoch: 459, Step 0, Loss: 7.962945346662309e-06
Epoch: 459, Step 10, Loss: 8.485992111673113e-06
Epoch: 459, Step 20, Loss: 9.554193638905417e-06
Test Accuracy: 48.79%

================ epoch: 460 ================
Epoch: 460, Step 0, Loss: 7.839896170480642e-06
Epoch: 460, Step 10, Loss: 8.355668796866667e-06
Epoch: 460, Step 20, Loss: 9.390727427671663e-06
Test Accuracy: 48.8%

================ epoch: 461 ================
Epoch: 461, Step 0, Loss: 7.717663720541168e-06
Epoch: 461, Step 10, Loss: 8.228314072766807e-06
Epoch: 461, Step 20, Loss: 9.23811148823006e-06
Test Accuracy: 48.78%

================ epoch: 462 ================
Epoch: 462, Step 0, Loss: 7.59

Test Accuracy: 48.809999999999995%

================ epoch: 496 ================
Epoch: 496, Step 0, Loss: 4.605767117027426e-06
Epoch: 496, Step 10, Loss: 4.931494913762435e-06
Epoch: 496, Step 20, Loss: 5.265310846880311e-06
Test Accuracy: 48.809999999999995%

================ epoch: 497 ================
Epoch: 497, Step 0, Loss: 4.540866029856261e-06
Epoch: 497, Step 10, Loss: 4.8629271987010725e-06
Epoch: 497, Step 20, Loss: 5.186117050470784e-06
Test Accuracy: 48.809999999999995%

================ epoch: 498 ================
Epoch: 498, Step 0, Loss: 4.47939919467899e-06
Epoch: 498, Step 10, Loss: 4.794942014996195e-06
Epoch: 498, Step 20, Loss: 5.1151782827218994e-06
Test Accuracy: 48.809999999999995%

================ epoch: 499 ================
Epoch: 499, Step 0, Loss: 4.415488547238056e-06
Epoch: 499, Step 10, Loss: 4.727072791865794e-06
Epoch: 499, Step 20, Loss: 5.0378453124722e-06
Test Accuracy: 48.809999999999995%
